<a href="https://colab.research.google.com/github/OptimumCoder/EVA4/blob/master/S4_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
      def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, 3),  # (3,28) > (3,26)
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2, 2),   # (3,26) > (3,13)
            nn.Conv2d(32, 16, 1), # (3,13) > (1,13)
            nn.Dropout(0.25)
            
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 3), # (1,13) > (3,11)
            nn.ReLU(),
            nn.BatchNorm2d(32),
            # nn.AvgPool2d(2, stride=2),
            nn.Conv2d(32, 32, 3), # (1,13) > (3,11)
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2, 2),   # (3,11) > (3,5)
            nn.Conv2d(32, 16, 1),
            nn.Dropout(0.25)
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(16, 16, 3), # (3,5) > (3,3)
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.25)
        )
        
        self.conv4 = nn.Conv2d(16, 10, 2) 
        # self.conv4 = nn.AvgPool2d(4)

        # self.conv4 = nn.Sequential(
        #     nn.AvgPool2d(2),
        #     nn.Conv2d(16,10,1)
        # )
                
        
      def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)   
        x = x.view(-1, 10)   
        return F.log_softmax(x, dim=1)


In [64]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
              ReLU-2           [-1, 32, 26, 26]               0
       BatchNorm2d-3           [-1, 32, 26, 26]              64
         MaxPool2d-4           [-1, 32, 13, 13]               0
            Conv2d-5           [-1, 16, 13, 13]             528
           Dropout-6           [-1, 16, 13, 13]               0
            Conv2d-7           [-1, 32, 11, 11]           4,640
              ReLU-8           [-1, 32, 11, 11]               0
       BatchNorm2d-9           [-1, 32, 11, 11]              64
           Conv2d-10             [-1, 32, 9, 9]           9,248
             ReLU-11             [-1, 32, 9, 9]               0
      BatchNorm2d-12             [-1, 32, 9, 9]              64
        MaxPool2d-13             [-1, 32, 4, 4]               0
           Conv2d-14             [-1, 1

In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [67]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)



  0%|          | 0/469 [00:00<?, ?it/s]

loss=2.446976900100708 batch_id=0:   0%|          | 0/469 [00:00<?, ?it/s]

loss=2.446976900100708 batch_id=0:   0%|          | 1/469 [00:00<00:51,  9.06it/s]

loss=2.4708352088928223 batch_id=1:   0%|          | 1/469 [00:00<00:51,  9.06it/s]

loss=2.3800981044769287 batch_id=2:   0%|          | 1/469 [00:00<00:51,  9.06it/s]

loss=2.4617691040039062 batch_id=3:   0%|          | 1/469 [00:00<00:51,  9.06it/s]

loss=2.4617691040039062 batch_id=3:   1%|          | 4/469 [00:00<00:41, 11.19it/s]

loss=2.3327560424804688 batch_id=4:   1%|          | 4/469 [00:00<00:41, 11.19it/s]

loss=2.258636474609375 batch_id=5:   1%|          | 4/469 [00:00<00:41, 11.19it/s] 

loss=2.222956418991089 batch_id=6:   1%|          | 4/469 [00:00<00:41, 11.19it/s]

loss=2.222956418991089 batch_id=6:   1%|▏         | 7/469 [00:00<00:34, 13.32it/s]

loss=2.1633968353271484 batch_id=7:   1%|▏         | 7/469 [00:00<00:34, 13.32it/s]

loss=2.209247589111328 batch_id=8:


Test set: Average loss: 0.0608, Accuracy: 9803/10000 (98%)





loss=0.0940384566783905 batch_id=0:   0%|          | 0/469 [00:00<?, ?it/s]

loss=0.06367778778076172 batch_id=1:   0%|          | 0/469 [00:00<?, ?it/s]

loss=0.06367778778076172 batch_id=1:   0%|          | 2/469 [00:00<00:28, 16.37it/s]

loss=0.15165328979492188 batch_id=2:   0%|          | 2/469 [00:00<00:28, 16.37it/s]

loss=0.07880081236362457 batch_id=3:   0%|          | 2/469 [00:00<00:28, 16.37it/s]

loss=0.12716610729694366 batch_id=4:   0%|          | 2/469 [00:00<00:28, 16.37it/s]

loss=0.11547873914241791 batch_id=5:   0%|          | 2/469 [00:00<00:28, 16.37it/s]

loss=0.11547873914241791 batch_id=5:   1%|▏         | 6/469 [00:00<00:24, 19.06it/s]

loss=0.1373225897550583 batch_id=6:   1%|▏         | 6/469 [00:00<00:24, 19.06it/s] 

loss=0.15574708580970764 batch_id=7:   1%|▏         | 6/469 [00:00<00:24, 19.06it/s]

loss=0.08127596974372864 batch_id=8:   1%|▏         | 6/469 [00:00<00:24, 19.06it/s]

loss=0.09437116980552673 batch_id=9:   1%|▏         | 6/469 [00:00<00


Test set: Average loss: 0.0382, Accuracy: 9879/10000 (99%)





loss=0.10552877187728882 batch_id=0:   0%|          | 0/469 [00:00<?, ?it/s]

loss=0.10552877187728882 batch_id=0:   0%|          | 1/469 [00:00<00:50,  9.21it/s]

loss=0.060139529407024384 batch_id=1:   0%|          | 1/469 [00:00<00:50,  9.21it/s]

loss=0.08794812858104706 batch_id=2:   0%|          | 1/469 [00:00<00:50,  9.21it/s] 

loss=0.06455960869789124 batch_id=3:   0%|          | 1/469 [00:00<00:50,  9.21it/s]

loss=0.046046238392591476 batch_id=4:   0%|          | 1/469 [00:00<00:50,  9.21it/s]

loss=0.046046238392591476 batch_id=4:   1%|          | 5/469 [00:00<00:39, 11.86it/s]

loss=0.05068032816052437 batch_id=5:   1%|          | 5/469 [00:00<00:39, 11.86it/s] 

loss=0.06452222168445587 batch_id=6:   1%|          | 5/469 [00:00<00:39, 11.86it/s]

loss=0.08569920808076859 batch_id=7:   1%|          | 5/469 [00:00<00:39, 11.86it/s]

loss=0.08569920808076859 batch_id=7:   2%|▏         | 8/469 [00:00<00:31, 14.47it/s]

loss=0.1103673204779625 batch_id=8:   2%|▏         | 8/

...


Test set: Average loss: 0.0189, Accuracy: 9947/10000 (99%)





loss=0.01214367151260376 batch_id=0:   0%|          | 0/469 [00:00<?, ?it/s]

loss=0.01872938498854637 batch_id=1:   0%|          | 0/469 [00:00<?, ?it/s]

loss=0.01872938498854637 batch_id=1:   0%|          | 2/469 [00:00<00:26, 17.80it/s]

loss=0.015365798026323318 batch_id=2:   0%|          | 2/469 [00:00<00:26, 17.80it/s]

loss=0.008968759328126907 batch_id=3:   0%|          | 2/469 [00:00<00:26, 17.80it/s]

loss=0.011443573981523514 batch_id=4:   0%|          | 2/469 [00:00<00:26, 17.80it/s]

loss=0.00971372053027153 batch_id=5:   0%|          | 2/469 [00:00<00:26, 17.80it/s] 

loss=0.00971372053027153 batch_id=5:   1%|▏         | 6/469 [00:00<00:22, 20.44it/s]

loss=0.035549458116292953 batch_id=6:   1%|▏         | 6/469 [00:00<00:22, 20.44it/s]

loss=0.03574904054403305 batch_id=7:   1%|▏         | 6/469 [00:00<00:22, 20.44it/s] 

loss=0.08207444846630096 batch_id=8:   1%|▏         | 6/469 [00:00<00:22, 20.44it/s]

loss=0.08207444846630096 batch_id=8:   2%|▏         | 9/469 [0


Test set: Average loss: 0.0192, Accuracy: 9946/10000 (99%)





loss=0.02726486325263977 batch_id=0:   0%|          | 0/469 [00:00<?, ?it/s]

loss=0.02726486325263977 batch_id=0:   0%|          | 1/469 [00:00<00:55,  8.48it/s]

loss=0.00947154313325882 batch_id=1:   0%|          | 1/469 [00:00<00:55,  8.48it/s]

loss=0.008493315428495407 batch_id=2:   0%|          | 1/469 [00:00<00:55,  8.48it/s]

loss=0.008753113448619843 batch_id=3:   0%|          | 1/469 [00:00<00:55,  8.48it/s]

loss=0.008753113448619843 batch_id=3:   1%|          | 4/469 [00:00<00:44, 10.44it/s]

loss=0.035445909947156906 batch_id=4:   1%|          | 4/469 [00:00<00:44, 10.44it/s]

loss=0.00647328794002533 batch_id=5:   1%|          | 4/469 [00:00<00:44, 10.44it/s] 

loss=0.002908002585172653 batch_id=6:   1%|          | 4/469 [00:00<00:44, 10.44it/s]

loss=0.002908002585172653 batch_id=6:   1%|▏         | 7/469 [00:00<00:36, 12.69it/s]

loss=0.010074831545352936 batch_id=7:   1%|▏         | 7/469 [00:00<00:36, 12.69it/s]

loss=0.04627664387226105 batch_id=8:   1%|▏         


Test set: Average loss: 0.0181, Accuracy: 9943/10000 (99%)

